In [ ]:
import os
import pandas as pd
import great_expectations as gx

from dotenv import load_dotenv
from google.cloud import storage

# Carrega variáveis de ambiente (se existirem)
load_dotenv()
context = gx.get_context()

# Regex para garantir que só existem dígitos (0-9). O padrão r"^\d+(\.0)?$" aceita "123" ou "123.0" (comum em floats do pandas)
NUMERIC_REGEX = r"^\d+(\.0)?$"
GCS_PATH = "gs://airbnb-datalake-jpcs/raw"
USE_CLOUD = False
BASE_PATH = "../data/raw" if not USE_CLOUD else GCS_PATH

print(f"Ambiente de Execução: {'CLOUD (GCP)' if USE_CLOUD else 'LOCAL (Mock)'}")
print(f"Caminho dos Dados: {BASE_PATH}")

🌍 Ambiente de Execução: LOCAL (Mock)
📂 Caminho dos Dados: ../data/silver


In [3]:
try:
    df_listings = pd.read_csv(f"{BASE_PATH}/dim_listings.csv", quotechar='"', on_bad_lines='warn', low_memory=False)
    df_reviews = pd.read_csv(f"{BASE_PATH}/fact_reviews.csv", quotechar='"', on_bad_lines='warn')
    print(f"✅ Listings carregado: {df_listings.shape}")
    print(f"✅ Reviews carregado: {df_reviews.shape}")
except FileNotFoundError as e:
    print(f"❌ Erro: Arquivo não encontrado em {BASE_PATH}. Verifique se moveu os CSVs para a pasta data/silver!")
    raise e

✅ Listings carregado: (38670, 11)
✅ Reviews carregado: (264561, 5)


In [4]:
def print_report(results, table_name):
    print(f"\n📊 RELATÓRIO: {table_name}")
    
    is_success = results.get("success") if isinstance(results, dict) else results.success
    print(f"Status Global: {'✅ APROVADO' if is_success else '❌ FALHOU (Esperado em dados Raw)'}")
    print("-" * 40)
    
    result_list = results.get("results") if isinstance(results, dict) else results.results
    
    for res in result_list:
        if isinstance(res, dict):
            success = res.get("success")
            config = res.get("expectation_config")
            result_detail = res.get("result")
        else:
            success = res.success
            config = res.expectation_config
            result_detail = res.result

        status = "✅" if success else "❌"
        if isinstance(config, dict):
             regra = config.get("expectation_type")
             coluna = config.get("kwargs", {}).get("column", "Table")
        else:
             regra = config.expectation_type
             coluna = config.kwargs.get("column", "Table")
        
        print(f"{status} [{coluna}] {regra}")
        
        if not success:
            unexpected_count = result_detail.get('unexpected_count') if isinstance(result_detail, dict) else result_detail.unexpected_count
            print(f"   ⚠️ Falhas encontradas: {unexpected_count} registros")


In [5]:
validator_listings = context.sources.pandas_default.read_dataframe(df_listings)

print("🛡️ Great Expectations inicializado com sucesso!")
print(f"Tipo do validador: {type(validator_listings)}")

# --- REGRAS PARA LISTINGS (Imóveis) ---
print("🔍 Validando Tabela: LISTINGS")

validator_listings.expect_column_values_to_not_be_null("SK_LISTING")
validator_listings.expect_column_values_to_be_unique("SK_LISTING")
validator_listings.expect_column_values_to_match_regex("SK_LISTING", regex=NUMERIC_REGEX)
validator_listings.expect_column_values_to_match_regex("SK_HOST", regex=NUMERIC_REGEX)
validator_listings.expect_column_values_to_not_be_null("VLR_DIARIA_BRL") 
validator_listings.expect_column_values_to_match_regex("VLR_DIARIA_BRL", regex=NUMERIC_REGEX)
validator_listings.expect_column_values_to_be_between("QTD_DIAS_DISPONIVEIS", min_value=0, max_value=365)

results_listings = validator_listings.validate()
print_report(results_listings, "Listings")

🛡️ Great Expectations inicializado com sucesso!
Tipo do validador: <class 'great_expectations.validator.validator.Validator'>
🔍 Validando Tabela: LISTINGS


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/28 [00:00<?, ?it/s]


📊 RELATÓRIO: Listings
Status Global: ✅ APROVADO
----------------------------------------
✅ [SK_LISTING] expect_column_values_to_not_be_null
✅ [SK_LISTING] expect_column_values_to_be_unique
✅ [SK_LISTING] expect_column_values_to_match_regex
✅ [SK_HOST] expect_column_values_to_match_regex
✅ [VLR_DIARIA_BRL] expect_column_values_to_not_be_null
✅ [VLR_DIARIA_BRL] expect_column_values_to_match_regex
✅ [QTD_DIAS_DISPONIVEIS] expect_column_values_to_be_between


In [ ]:

validator_reviews = context.sources.pandas_default.read_dataframe(df_reviews)

# --- REGRAS PARA REVIEWS (Avaliações) ---
print("\n🔍 Validando Tabela: REVIEWS")

validator_reviews.expect_column_values_to_match_regex("SK_REVIEW", regex=NUMERIC_REGEX)
validator_reviews.expect_column_values_to_not_be_null("SK_LISTING")
validator_reviews.expect_column_values_to_match_regex("SK_LISTING", regex=NUMERIC_REGEX)
validator_reviews.expect_column_to_exist("NM_REVIEWER")

results_reviews = validator_reviews.validate()
print_report(results_reviews, "Reviews")